## GAN의 전성시대를 연 DCGAN(Deep Convolutional GAN)
GAN은 학습이 불안정하기로 악명이 높다. 학습이 어렵다는 점은 GAN 모델이 다양한 곳에 응용되는 것을 가로막는 큰 장애물이었다. 이런 상황에서 수많은 실험 끝에 안정적인 학습이 가능한 GAN 모델의 구조를 찾아낸 것이 DCGAN이다.

DCGAN의 특징은 몇 가지로 요약할 수 있다. 먼저, 선형 레이어와 풀링 레이어(Pooling Layer)를 최대한 배제하고 합성곱(Convolution)과 ‘Transposed Convolution(Fractional-Strided Convolution)’으로 네트워크 구조를 만들었다. 풀링 레이어는 여러 딥러닝 모델에서 불필요한 매개변수의 수를 줄이고 중요한 특징만을 골라내는 역할을 하는 레이어지만 이미지의 위치 정보를 잃어버린다는 단점이 있다. 이미지를 생성하기 위해서는 위치 정보가 중요하기 때문에 DCGAN은 풀링 레이어를 배제했다. 선형 레이어 역시 마찬가지로 위치 정보를 잃어버리므로 모델의 깊은 레이어에서는 선형 레이어를 사용하지 않았다.

DCGAN의 또 다른 특징은 배치 정규화(Batch Normalization)를 사용했다는 점이다. 배치 정규화는 레이어의 입력 데이터 분포가 치우쳐져 있을 때 평균과 분산을 조정해주는 역할을 한다. 이는 역전파가 각 레이어에 쉽게 전달되도록 해 학습이 안정적으로 이뤄지도록 돕는 데 중요한 역할을 한다.

이외에도 DCGAN은 수많은 실험을 통해 GAN을 학습시키는 가장 좋은 조건들을 찾아냈다. DCGAN은 마지막 레이어를 제외하고 생성자의 모든 레이어에 ReLU를 사용했고, 구분자의 모든 레이어에 LeakyReLU를 사용했다. 또한, 가장 좋은 최적화 기법과 적절한 학습 속도(Learning Rate) 등을 찾아내기도 했다.

DCGAN의 성공은 GAN 모델이 유명해지는 데 결정적인 역할을 했다. DCGAN에서 사용한 모델 구조는 아직도 새로운 GAN 모델을 설계할 때 베이스 모델이 되고 있다.

DCGAN의 네트워크 구조는 기존 GAN에서 생성자와 구분자만 교체하는 것만으로 간단히 구현할 수 있다. DCGAN의 생성자는 GAN과 마찬가지로 랜덤 벡터 z를 받고 가짜 이미지를 생성하는 함수다. 다만 그 구현에서 ‘Transposed Convolution’과 배치 정규화 등을 사용한다는 점이 다르다.

## <코드9> DCGAN의 생성자

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.utils import save_image

/Users/janghan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
class Generator(nn.Module):
    
  # 네트워크 구조
  def __init__(self):
    super(Generator, self).__init__()
    self.main = nn.Sequential(
      nn.ConvTranspose2d(in_channels=100, out_channels=28*8, 
        kernel_size=7, stride=1, padding=0, 
        bias=False),
      nn.BatchNorm2d(num_features=28*8),
      nn.ReLU(inplace=True),
      nn.ConvTranspose2d(in_channels=28*8, out_channels=28*4, 
        kernel_size=4, stride=2, padding=1, 
        bias=False),
      nn.BatchNorm2d(num_features=28*4),
      nn.ReLU(True),
      nn.ConvTranspose2d(in_channels=28*4, out_channels=1, 
        kernel_size=4, stride=2, padding=1, 
        bias=False),
      nn.Tanh()
    )
        
  # (batch_size x 100) 크기의 랜덤 벡터를 받아 
  # 이미지를 (batch_size x 1 x 28 x 28) 크기로 출력한다.
  def forward(self, inputs):
    inputs = inputs.view(-1, 100, 1, 1)
    return self.main(inputs)

DCGAN의 구분자도 GAN의 구분자와 입력과 출력이 동일하다. 단지 convolution 레이어와 배치 정규화 등을 사용한다는 차이점만 있다.

## <코드10> DCGAN의 구분자

In [6]:
class Discriminator(nn.Module):
    
  # 네트워크 구조
  def __init__(self):
    super(Discriminator, self).__init__()
    self.main = nn.Sequential(
      nn.Conv2d(in_channels=1, out_channels=28*4, 
        kernel_size=4, stride=2, padding=1, 
        bias=False),
      nn.BatchNorm2d(num_features=28*4),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Conv2d(in_channels=28*4, out_channels=28*8, 
        kernel_size=4, stride=2, padding=1, 
        bias=False),
      nn.BatchNorm2d(num_features=28*8),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Conv2d(in_channels=28*8, out_channels=1, 
        kernel_size=7, stride=1, padding=0, 
        bias=False),
      nn.Sigmoid()
    )
        
  # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아
  # 이미지가 진짜일 확률을 0~1 사이로 출력한다.
  def forward(self, inputs):
    o = self.main(inputs)
    return o.view(-1, 1)